In [38]:
import logging
import pandas as pd
import re

def normalize_name(name):
    # Remove suffixes like 'Jr.', 'Sr.', 'III', etc.
    name = re.sub(r'\s+(Jr\.|Sr\.|I{2,}|IV)$', '', name)
    # Remove middle initials
    name = re.sub(r'\s+[A-Z]\.\s+', ' ', name)
    # Remove any remaining periods and convert to lowercase
    return name.replace('.', '').lower().strip()

def merge_dataframes(fanduel_df, predictions_df):
    # Normalize names in both dataframes
    fanduel_df['Normalized_Name'] = fanduel_df['Nickname'].apply(normalize_name)
    predictions_df['Normalized_Name'] = predictions_df['player_name'].apply(normalize_name)
    
    # Merge dataframes based on normalized names
    merged_df = pd.merge(fanduel_df, predictions_df, on='Normalized_Name', how='left')
    
    # Drop the temporary normalized name columns
    merged_df = merged_df.drop(columns=['Normalized_Name'])
    
    return merged_df

# Load the data
logging.info("Loading data from CSV...")
qbpredictions_df = pd.read_csv('predictions/QB_predictions_week3_season2024.csv')
rbpredictions_df = pd.read_csv('predictions/RB_predictions_week3_season2024.csv')
tepredictions_df = pd.read_csv('predictions/TE_predictions_week3_season2024.csv')
wrpredictions_df = pd.read_csv('predictions/WR_predictions_week3_season2024.csv')
fanduel_df = pd.read_csv('fanduel/FanDuel-NFL-2024 MST-09 MST-22 MST-107027-players-list.csv')

# Combine all prediction dataframes
all_predictions = pd.concat([qbpredictions_df, rbpredictions_df, tepredictions_df, wrpredictions_df], ignore_index=True)

# Merge FanDuel data with predictions
df = merge_dataframes(fanduel_df, all_predictions)

# Calculate value
df['value'] = (df['fanduel_fantasy_points'] / df['Salary']) * 1000

# Sort and save to CSV
df.sort_values(['value'], ascending=False).to_csv('predictions/fanduel_value.csv', index=False)

# Display the resulting dataframe
print(df)



               Id Position   First Name           Nickname  Last Name   FPPG  \
0    107027-86631       WR       CeeDee        CeeDee Lamb       Lamb  14.05   
1    107027-85671       WR       Justin   Justin Jefferson  Jefferson  19.10   
2    107027-86997       WR      Amon-Ra  Amon-Ra St. Brown  St. Brown  11.60   
3    107027-53681       WR       Tyreek        Tyreek Hill       Hill  15.30   
4    107027-63115       QB        Lamar      Lamar Jackson    Jackson  22.75   
..            ...      ...          ...                ...        ...    ...   
875  107027-12538        D  Los Angeles   Los Angeles Rams       Rams   1.50   
876  107027-12543        D     New York    New York Giants     Giants   5.00   
877  107027-12531        D       Denver     Denver Broncos    Broncos   8.00   
878  107027-12525        D      Atlanta    Atlanta Falcons    Falcons   3.00   
879  107027-12553        D     Carolina  Carolina Panthers   Panthers   2.00   

     Played  Salary     Game Team  ... 

In [32]:
from nfl_data_py import import_weekly_data, import_players, import_weekly_rosters

In [25]:
df = import_weekly_data([2024])

Downcasting floats.


In [26]:
df

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,8.58,8.58
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,15.14,15.14
2,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,LA,2024,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.68,14.68
3,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,LA,2024,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,6.64,6.64
4,00-0027973,A.Dalton,Andy Dalton,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,CAR,2024,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,00-0039919,R.Odunze,Rome Odunze,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,CHI,2024,1,REG,...,1.0,-2.087302,0,0.500000,0.137931,0.102804,0.278859,0.0,1.10,2.10
615,00-0039919,R.Odunze,Rome Odunze,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,CHI,2024,2,REG,...,1.0,-2.121063,0,0.523810,0.147059,0.254032,0.398411,0.0,3.30,5.30
616,00-0039920,M.Corley,Malachi Corley,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,...,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,0.0,0.40,1.40
617,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,1,REG,...,0.0,-0.088616,0,-1.250000,0.032258,-0.020942,0.033727,0.0,1.80,2.80


In [27]:
df.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [29]:
pdf = import_players()
pdf

,status,display_name,first_name,last_name,esb_id,gsis_id,birth_date,college_name,position_group,position,...,college_conference,status_description_abbr,status_short_description,gsis_it_id,short_name,smart_id,headshot,suffix,uniform_number,draft_round
0,RET,'Omar Ellison,'Omar,Ellison,ELL711319,00-0004866,1971-10-08,None,WR,WR,...,None,None,None,NaN,None,3200454c-4c71-1319-728e-d49d3d236f8f,None,None,None,None
1,ACT,A'Shawn Robinson,A'Shawn,Robinson,ROB367960,00-0032889,1995-03-21,Alabama,DL,DT,...,Southeastern Conference,A01,Active,43335.0,A.Robinson,3200524f-4236-7960-bf20-bc060ac0f49c,https://static.www.nfl.com/image/private/f_aut...,None,94,None
2,DEV,A.J. Arcuri,A.J.,Arcuri,ARC716900,00-0037845,1997-08-13,Michigan State,OL,T,...,Big Ten Conference,P01,Practice Squad,54726.0,A.Arcuri,32004152-4371-6900-5185-8cdd66b2ad11,None,None,61,None
3,ACT,A.J. Barner,A.J.,Barner,BAR235889,00-0039793,2002-05-03,Michigan,TE,TE,...,Big Ten Conference,A01,Active,57242.0,A.Barner,32004241-5223-5889-95d9-0ba3aeeb36ed,https://static.www.nfl.com/image/upload/f_auto...,None,88,None
4,RES,A.J. Bouye,Arlandus,Bouye,BOU651714,00-0030228,1991-08-16,Central Florida,DB,CB,...,American Athletic Conference,R01,R/Injured,40688.0,A.Bouye,3200424f-5565-1714-cb38-07c822111a12,https://static.www.nfl.com/image/private/f_aut...,None,24,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20748,CUT,Zuri Henry,Zuri,Henry,HEN713594,00-0039689,2000-04-12,Texas-El Paso,OL,T,...,Conference USA,W03,Waivers/No Rec.,57839.0,Z.Henry,32004845-4e71-3594-d6f6-5f3aaca3acfc,None,None,65,None
20749,RET,Zuriel Smith,Zuriel,Smith,SMI828252,00-0022024,1980-01-15,None,WR,WR,...,None,None,None,NaN,None,3200534d-4982-8252-c800-85b370612021,None,None,None,None
20750,CUT,Zurlon Tipton,Zurlon,Tipton,TIP645432,00-0030855,1989-04-27,Central Michigan,RB,RB,...,Mid-American Conference,A01,Active,41693.0,None,32005449-5064-5432-923a-94c7fae9b469,https://static.www.nfl.com/image/private/f_aut...,None,None,None
20751,DEV,Zyon Gilbert,Zyon,Gilbert,GIL144859,00-0037373,1999-02-04,Florida Atlantic,DB,CB,...,Conference USA,P01,Practice Squad,54958.0,Z.Gilbert,32004749-4c14-4859-6e4a-327bb4fcec5a,https://static.www.nfl.com/image/private/f_aut...,None,42,None


In [30]:
pdf.columns

Index(['status', 'display_name', 'first_name', 'last_name', 'esb_id',
       'gsis_id', 'birth_date', 'college_name', 'position_group', 'position',
       'jersey_number', 'height', 'weight', 'years_of_experience', 'team_abbr',
       'team_seq', 'current_team_id', 'football_name', 'entry_year',
       'rookie_year', 'draft_club', 'draft_number', 'college_conference',
       'status_description_abbr', 'status_short_description', 'gsis_it_id',
       'short_name', 'smart_id', 'headshot', 'suffix', 'uniform_number',
       'draft_round'],
      dtype='object')

In [33]:
wf = import_weekly_rosters([2024])
wf

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,...,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
0,2024,NYJ,QB,QB,8.0,ACT,Aaron Rodgers,Aaron,Rodgers,1983-12-02,...,A01,Aaron,ROD339293,29851,3200524f-4433-9293-a3cf-ad7758d03003,2005,2005.0,GB,24.0,40.789
1,2024,NYJ,QB,QB,8.0,ACT,Aaron Rodgers,Aaron,Rodgers,1983-12-02,...,A01,Aaron,ROD339293,29851,3200524f-4433-9293-a3cf-ad7758d03003,2005,2005.0,GB,24.0,40.799
2,2024,NYJ,QB,QB,8.0,ACT,Aaron Rodgers,Aaron,Rodgers,1983-12-02,...,A01,Aaron,ROD339293,29851,3200524f-4433-9293-a3cf-ad7758d03003,2005,2005.0,GB,24.0,40.772
3,2024,SEA,P,P,9.0,RET,Jon Ryan,Jon,Ryan,1981-11-26,...,R02,Jon,RYA229481,30714,32005259-4122-9481-61e2-b493c152ed78,2006,2006.0,None,NaN,42.823
4,2024,SEA,P,P,9.0,RET,Jon Ryan,Jon,Ryan,1981-11-26,...,R02,Jon,RYA229481,30714,32005259-4122-9481-61e2-b493c152ed78,2006,2006.0,None,NaN,42.804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8193,2024,NE,DL,DE,30.0,CUT,Christian McCarroll,Christian,McCarroll,NaT,...,W03,Christian,MCC175443,58058,32004d43-4317-5443-a1e6-84b5eefecfbe,2024,2024.0,None,NaN,NaN
8194,2024,CAR,LB,LB,41.0,CUT,Aaron Beasley,Aaron,Beasley,2000-10-03,...,W03,Aaron,BEA416040,58060,32004245-4141-6040-6ad9-e8707642b097,2024,2024.0,None,NaN,23.932
8195,2024,NO,DL,DT,76.0,RES,Camron Peterson,Camron,Peterson,1999-09-13,...,R01,Camron,PAT382575,58061,32005041-5438-2575-6a39-c3744d58b516,2024,2024.0,None,NaN,25.008
8196,2024,NO,DL,DT,76.0,RES,Camron Peterson,Camron,Peterson,1999-09-13,...,R01,Camron,PAT382575,58061,32005041-5438-2575-6a39-c3744d58b516,2024,2024.0,None,NaN,24.988


In [34]:
wf.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week',
       'game_type', 'status_description_abbr', 'football_name', 'esb_id',
       'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club',
       'draft_number', 'age'],
      dtype='object')

In [36]:
wf.to_csv('weekly_rosters.csv')